#### PROBLEMA 11
    Write a function inverse_quantum_fourier_transform(num_qubits, shots, seed, input_state) that creates and simulates an inverse Quantum Fourier Transform (QFT) circuit. The function should:
    - Initialize the qubits with the provided input_state.
    - Apply the inverse QFT to the qubits.
    - Include the final swap to the algorithm
    - Perform the specified number of shots measurements.
    - Use the given seed for reproducibility.
    
    The function should return a list of measurement outcomes from the simulation (counts).


In [ ]:
!pip install numpy
!pip install qiskit
!pip install qiskit_aer

In [39]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit.circuit.library import Initialize

def apply_inverse_qft(circuit, num_qubits):
    
    
    # Reverse the order of the qubits
    for i in range(num_qubits // 2):
        circuit.swap(i, num_qubits - i - 1)
        # back iteration and negative angle for inverse
    for qubit in range(num_qubits - 1, -1, -1): 
        for control_qubit in range(num_qubits - 1, qubit, -1):
            angle = -np.pi / (2 ** (control_qubit - qubit))
            circuit.cp(angle, control_qubit, qubit)
        
        circuit.h(qubit)

def inverse_quantum_fourier_transform(num_qubits, shots, seed, input_state):
    qc =QuantumCircuit(num_qubits)
    qc.initialize(Statevector(input_state),range(num_qubits)) 
    apply_inverse_qft(qc, num_qubits)
    qc.measure_all()
    
    backend = AerSimulator()
    
    transpiled_qc = transpile(qc, backend)

    job = backend.run(transpiled_qc, shots=shots, seed_simulator=seed)

    result = job.result()
    counts = result.get_counts(transpiled_qc)
    return counts
    


In [40]:
exp1 = {'1': 479, '0': 521}
state1 = [1, 0]
out1 = inverse_quantum_fourier_transform(1, 1000, 19, state1)
assert exp1 == out1, f"Test failed"

In [41]:
    num_qubits = 2
    input_state = [0, 1, 0, 0]
    qc =QuantumCircuit(num_qubits)
    qc.initialize(Statevector(input_state), range(num_qubits))
    apply_inverse_qft(qc, num_qubits)
    qc.measure_all()
    
    backend = AerSimulator(seed_simulator=19)
    
    transpiled_qc = transpile(qc, backend)

    job = backend.run(transpiled_qc, shots=1000)

    result = job.result()
    counts = result.get_counts(transpiled_qc)
    print(counts)
   

{'01': 255, '10': 227, '00': 266, '11': 252}


{'0': 521, '1': 479}
